# 读取Word文字

In [1]:
!pip install python-docx

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [13]:
import docx

path = 'data/111.docx'
file = docx.Document(path)

paragraphs 只包含正常文字部分，不包含图片和表格，但是图片的名称会保留下来，如“图1- 1   毕业生规模及性别分布”

结构大体如下：
```text
paragraphs
|
|- para
|   |
|   |- runs
|   |   |
|   |   |- run
|   |   |   |
|   |   |   |- font
|   |   |   |   |- size 
|   |   |   |   |- name
|   |   |   |
|   |   |   |- style
|   |   |   |   |
|   |   |   |   |- font
|   |   |   |   |   |- size
|   |   |   |   |   |- name
|   |- style
|   |   |- font
|   |   |   |- size  #目前使用中，这个会生效，其余部分极大可能是None
```


In [11]:
for para in file.paragraphs:
    if para.runs.__len__() == 0:
        continue
    print(f'{para.style.font.size}\n{para.text}', end='\n---\n')

133350

---
133350
2023届毕业生就业质量年度报告
---
133350
	
---
133350
二〇二三年十二月
---
133350

---
133350

---
279400
学校简介
---
None
湖南大众传媒职业技术学院是在原湖南银行学校和湖南教育电视台的基础上，按照“前台后院”模式组建的一所公立传媒类高职学院，于2000年7月正式建立。2002年，原长沙县教师进修学校并入；2004年，原湖南省广播电视学校整体并入。学院实行由湖南省教育厅、湖南省广播电视局、湖南广播电视台共建，湖南省教育厅主管的管理体制。现为国家首批骨干高职院校、湖南省首批卓越高职院校、国家汉办设立在湖南的“国际汉语言文化传播基地”、马尔代夫维拉学院汉语中心（孔子学院）中方承办院校、湖南广播电视台节目生产基地，被誉为“广电湘军”的摇篮。
---
None
学院坚持“立足湖南、面向全国，服务文化产业、突出传媒特色”的办学定位，秉承“创意点亮人生”的校训，坚持“传播正能量”的学院精神，实施“引台兴校、原创强校、服务荣校”的发展战略，致力于为湖南“三高四新”战略实施、文化强省建设、传媒产业发展培养综合素质高、传播沟通能力强的创意型技术技能人才。现设有34个专业，基本形成了以“媒介内容生产”为核心，以“传媒技术”和“传媒管理”为支撑，兼及“文化教育”的专业发展格局，重点建设新闻出版与广播影视、动漫与艺术设计、新媒体技术三大特色专业群，面向全国28个省、自治区、直辖市招生，现有在校学生11000余名。
---
None
学院设新闻与传播学院、影视艺术学院、视觉艺术学院、新媒体技术学院、管理学院、国际传播学院6个二级学院和思想政治课教学部、体育课教学部2个教学部，实行校院（部）两级管理。现有教职员工683人，其中高级职称教师198人，博士、硕士431人。
---
None
学院主校区位于长沙市星沙国家级经济技术开发区，与湖南广电中心毗邻，是一个新建的环保型智能化绿色校园，获得了“湖南省文明单位”和“湖南省园林式单位”称号。学院是全国广播电视编辑记者、播音员主持人资格考试考点，雅思（IELTS）考试考点。
---
133350

---
133350

---
279400
编制说明
---
None
就业质量是检验学校育人成效的重要标准之一，也是学校办学质量的重要体现。为

## 将word内容解析成树型结构

创建一个虚拟的根节点，认为这个字体的要求为无穷大。然后所有节点都会链接一个比他大的节点，如果当前节点没有比他大的节点，则将质证向上移动到满足条件的节点。

In [12]:
import uuid


class Node:  # Define a generic class Node with type variable T
    def __init__(self, tag, value) -> None:
        self.tag = tag
        self.value = value
        self.children: list[Node] = []
        self.parent: Node | None = None
        self.uuid: str = str(uuid.uuid4())

    def add_child(self, child):
        child.parent = self
        self.children.append(child)
        return child


root = Node(float('inf'), None)

point = root
for para in file.paragraphs:
    if para.runs.__len__() == 0 or para.text.strip() == '':
        continue
    tag = para.style.font.size or 0
    while point.tag <= tag:
        point = point.parent
    point = point.add_child(Node(tag, para.text))


## 嵌入模型写入向量数据库

In [6]:

from qdrant_client import QdrantClient, models

collection_name = 'word_reader_test_1024'
qdrant_client = QdrantClient('http://192.168.100.111:6333')
vectors_config = models.VectorParams(size=1024, distance=models.Distance.COSINE)
qdrant_client.recreate_collection(collection_name, vectors_config)


/tmp/ipykernel_73680/3513621198.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(collection_name, vectors_config)


True

In [7]:
from langchain_qdrant import Qdrant
from langchain_community.embeddings import XinferenceEmbeddings

embeddings = XinferenceEmbeddings(server_url="http://192.168.100.111:9997",
                                  model_uid="bge-m3")

qdrant = Qdrant(client=qdrant_client, collection_name=collection_name, embeddings=embeddings)

In [8]:
# 遍历叶子结点，将内容写入向量数据库
file_name = '湖南大众传媒职业技术学院2023届毕业生就业质量年度报告'


class Count:
    idx = 0


def ite(node: Node):
    if node.children.__len__() == 0:
        parent_data = []
        _point = node
        while _point.parent:
            if _point.parent.value:
                parent_data.append(_point.parent.value)
            _point = _point.parent

        parent_data.append(file_name)
        parent_data.reverse()
        text = '\n'.join(parent_data) + '\n' + node.value
        print(f'\ntext: {text} \n---\n')
        qdrant.add_texts([text], metadatas=[{'parent': node.parent.uuid, 'idx': Count.idx}])
        Count.idx += 1
        return
    for child in node.children:
        ite(child)


ite(root)


text: 湖南大众传媒职业技术学院2023届毕业生就业质量年度报告
2023届毕业生就业质量年度报告 
---


text: 湖南大众传媒职业技术学院2023届毕业生就业质量年度报告
二〇二三年十二月 
---


text: 湖南大众传媒职业技术学院2023届毕业生就业质量年度报告
学校简介
湖南大众传媒职业技术学院是在原湖南银行学校和湖南教育电视台的基础上，按照“前台后院”模式组建的一所公立传媒类高职学院，于2000年7月正式建立。2002年，原长沙县教师进修学校并入；2004年，原湖南省广播电视学校整体并入。学院实行由湖南省教育厅、湖南省广播电视局、湖南广播电视台共建，湖南省教育厅主管的管理体制。现为国家首批骨干高职院校、湖南省首批卓越高职院校、国家汉办设立在湖南的“国际汉语言文化传播基地”、马尔代夫维拉学院汉语中心（孔子学院）中方承办院校、湖南广播电视台节目生产基地，被誉为“广电湘军”的摇篮。 
---


text: 湖南大众传媒职业技术学院2023届毕业生就业质量年度报告
学校简介
学院坚持“立足湖南、面向全国，服务文化产业、突出传媒特色”的办学定位，秉承“创意点亮人生”的校训，坚持“传播正能量”的学院精神，实施“引台兴校、原创强校、服务荣校”的发展战略，致力于为湖南“三高四新”战略实施、文化强省建设、传媒产业发展培养综合素质高、传播沟通能力强的创意型技术技能人才。现设有34个专业，基本形成了以“媒介内容生产”为核心，以“传媒技术”和“传媒管理”为支撑，兼及“文化教育”的专业发展格局，重点建设新闻出版与广播影视、动漫与艺术设计、新媒体技术三大特色专业群，面向全国28个省、自治区、直辖市招生，现有在校学生11000余名。 
---


text: 湖南大众传媒职业技术学院2023届毕业生就业质量年度报告
学校简介
学院设新闻与传播学院、影视艺术学院、视觉艺术学院、新媒体技术学院、管理学院、国际传播学院6个二级学院和思想政治课教学部、体育课教学部2个教学部，实行校院（部）两级管理。现有教职员工683人，其中高级职称教师198人，博士、硕士431人。 
---


text: 湖南大众传媒职业技术学院2023届毕业生就业质量年度报告
学校简介
学院主校区位于长沙市星沙国家级经济技术开发区，与湖南广电中心毗邻，是一个新建的环保型智能化绿色校园，获得了“湖南

# 读取Word表格

In [1]:
from docx import Document

doc = Document('data/111.docx')

输出所有表格，观察表格结构，发现合并的单元格会被使用多次，这正好也能够保证其正确的含义

In [2]:
for i, table in enumerate(doc.tables):
    print(f'第{i}个表格')
    for row in table.rows:
        print([cell.text for cell in row.cells])
    print('-' * 50)

第0个表格
['学院', '人数', '占比']
['新闻与传播学院', '820', '21.80%']
['视觉艺术学院', '804', '21.38%']
['新媒体技术学院', '714', '18.98%']
['管理学院', '674', '17.92%']
['影视艺术学院', '398', '10.58%']
['国际传播学院', '351', '9.33%']
['总计', '3761', '100.00%']
--------------------------------------------------
第1个表格
['学院', '专业', '人数', '占比']
['国际传播学院', '商务英语', '156', '4.15%']
['国际传播学院', '文秘', '118', '3.14%']
['国际传播学院', '休闲服务与管理', '77', '2.05%']
['国际传播学院 汇总', '国际传播学院 汇总', '351', '9.33%']
['新媒体技术学院', '数字媒体应用技术', '182', '4.84%']
['新媒体技术学院', '软件技术', '125', '3.32%']
['新媒体技术学院', '电子商务技术', '115', '3.06%']
['新媒体技术学院', '移动互联应用技术', '110', '2.92%']
['新媒体技术学院', '云计算技术与应用', '94', '2.50%']
['新媒体技术学院', '计算机网络技术', '88', '2.34%']
['新媒体技术学院 汇总', '新媒体技术学院 汇总', '714', '18.98%']
['管理学院', '会计', '179', '4.76%']
['管理学院', '电子商务', '175', '4.65%']
['管理学院', '传播与策划', '147', '3.91%']
['管理学院', '文化创意与策划', '88', '2.34%']
['管理学院', '金融管理', '85', '2.26%']
['管理学院 汇总', '管理学院 汇总', '674', '17.92%']
['视觉艺术学院', '动漫设计', '115', '3.06%']
['视觉艺术学院', '室内艺术设计', '114', '3.03%'

## 将表格转换为markdown格式


In [3]:
def convert_table_to_markdown(table):
    markdown = "|"
    for cell in table.rows[0].cells:
        markdown += f"{cell.text}|"
    markdown += "\n|"
    for cell in table.rows[0].cells:
        markdown += "---|"
    markdown += "\n"
    for row in table.rows[1:]:
        markdown += "|"
        for cell in row.cells:
            markdown += f" {cell.text} |"
        markdown += "\n"
    return markdown

In [4]:
markdown = convert_table_to_markdown(doc.tables[0])
print(markdown)

|学院|人数|占比|
|---|---|---|
| 新闻与传播学院 | 820 | 21.80% |
| 视觉艺术学院 | 804 | 21.38% |
| 新媒体技术学院 | 714 | 18.98% |
| 管理学院 | 674 | 17.92% |
| 影视艺术学院 | 398 | 10.58% |
| 国际传播学院 | 351 | 9.33% |
| 总计 | 3761 | 100.00% |



Markdown格式如下：

|学院|人数|占比|
|---|---|---|
| 新闻与传播学院 | 820 | 21.80% |
| 视觉艺术学院 | 804 | 21.38% |
| 新媒体技术学院 | 714 | 18.98% |
| 管理学院 | 674 | 17.92% |
| 影视艺术学院 | 398 | 10.58% |
| 国际传播学院 | 351 | 9.33% |
| 总计 | 3761 | 100.00% |

## 获取表格和文字的相对应用关系

需要获取文字、图片和表格的对应关系，我们需要得到全文的对象排列。

结构大体如下：
```text
paragraphs
|
|-element
    |
    |-body
        |
        |-inner_content_elements: 所有元素排列，一个列表
            |
            |-element: 每个元素，包含表格、文字等

```


### 统计结构中的类型分布

In [5]:
from collections import Counter

# 建立一个计数器
counter = Counter()
for element in doc.element.body.inner_content_elements:
    counter[element.__class__] += 1
counter

Counter({docx.oxml.text.paragraph.CT_P: 589, docx.oxml.table.CT_Tbl: 18})

### 获取表格和文字的相对应用关系

In [6]:
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl
import uuid


class Node:  # Define a generic class Node with type variable T
    def __init__(self, tag, value) -> None:
        self.tag = tag
        self.value = value
        self.children: list[Node] = []
        self.parent: Node | None = None
        self.uuid: str = str(uuid.uuid4())

    def add_child(self, child):
        child.parent = self
        self.children.append(child)
        return child


root = Node(float('inf'), None)
point = root

p_i = -1
t_i = -1

for element in doc.element.body.inner_content_elements:
    if isinstance(element, CT_P):
        p_i += 1
        para = doc.paragraphs[p_i]
        if para.runs.__len__() == 0 or para.text.strip() == '':
            continue
        tag = para.style.font.size or 0
        while point.tag <= tag:
            point = point.parent
        point = point.add_child(Node(tag, para.text))
    elif isinstance(element, CT_Tbl):
        t_i += 1
        table = doc.tables[t_i]
        text = convert_table_to_markdown(table)
        tag = 0
        point = point.add_child(Node(tag, text))

## 遍历叶子结点，将内容写入向量数据库

In [7]:

from qdrant_client import QdrantClient, models

collection_name = 'word_reader_test_1024'
qdrant_client = QdrantClient('http://192.168.100.111:6333')
vectors_config = models.VectorParams(size=1024, distance=models.Distance.COSINE)
qdrant_client.recreate_collection(collection_name, vectors_config)

from langchain_qdrant import Qdrant
from langchain_community.embeddings import XinferenceEmbeddings

embeddings = XinferenceEmbeddings(server_url="http://192.168.100.111:9997",
                                  model_uid="bge-m3")

qdrant = Qdrant(client=qdrant_client, collection_name=collection_name, embeddings=embeddings)
# 遍历叶子结点，将内容写入向量数据库
file_name = '湖南大众传媒职业技术学院2023届毕业生就业质量年度报告'


class Count:
    idx = 0


def ite(node: Node):
    if node.children.__len__() == 0:
        parent_data = []
        _point = node
        while _point.parent:
            if _point.parent.value:
                parent_data.append(_point.parent.value)
            _point = _point.parent

        parent_data.append(file_name)
        parent_data.reverse()
        text = '\n'.join(parent_data) + '\n' + node.value
        print(f'\ntext: {text} \n---\n')
        qdrant.add_texts([text], metadatas=[{'parent': node.parent.uuid, 'idx': Count.idx}])
        Count.idx += 1
        return
    for child in node.children:
        ite(child)


ite(root)

/tmp/ipykernel_11888/1033867827.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(collection_name, vectors_config)



text: 湖南大众传媒职业技术学院2023届毕业生就业质量年度报告
2023届毕业生就业质量年度报告 
---


text: 湖南大众传媒职业技术学院2023届毕业生就业质量年度报告
二〇二三年十二月 
---


text: 湖南大众传媒职业技术学院2023届毕业生就业质量年度报告
学校简介
湖南大众传媒职业技术学院是在原湖南银行学校和湖南教育电视台的基础上，按照“前台后院”模式组建的一所公立传媒类高职学院，于2000年7月正式建立。2002年，原长沙县教师进修学校并入；2004年，原湖南省广播电视学校整体并入。学院实行由湖南省教育厅、湖南省广播电视局、湖南广播电视台共建，湖南省教育厅主管的管理体制。现为国家首批骨干高职院校、湖南省首批卓越高职院校、国家汉办设立在湖南的“国际汉语言文化传播基地”、马尔代夫维拉学院汉语中心（孔子学院）中方承办院校、湖南广播电视台节目生产基地，被誉为“广电湘军”的摇篮。 
---


text: 湖南大众传媒职业技术学院2023届毕业生就业质量年度报告
学校简介
学院坚持“立足湖南、面向全国，服务文化产业、突出传媒特色”的办学定位，秉承“创意点亮人生”的校训，坚持“传播正能量”的学院精神，实施“引台兴校、原创强校、服务荣校”的发展战略，致力于为湖南“三高四新”战略实施、文化强省建设、传媒产业发展培养综合素质高、传播沟通能力强的创意型技术技能人才。现设有34个专业，基本形成了以“媒介内容生产”为核心，以“传媒技术”和“传媒管理”为支撑，兼及“文化教育”的专业发展格局，重点建设新闻出版与广播影视、动漫与艺术设计、新媒体技术三大特色专业群，面向全国28个省、自治区、直辖市招生，现有在校学生11000余名。 
---


text: 湖南大众传媒职业技术学院2023届毕业生就业质量年度报告
学校简介
学院设新闻与传播学院、影视艺术学院、视觉艺术学院、新媒体技术学院、管理学院、国际传播学院6个二级学院和思想政治课教学部、体育课教学部2个教学部，实行校院（部）两级管理。现有教职员工683人，其中高级职称教师198人，博士、硕士431人。 
---


text: 湖南大众传媒职业技术学院2023届毕业生就业质量年度报告
学校简介
学院主校区位于长沙市星沙国家级经济技术开发区，与湖南广电中心毗邻，是一个新建的环保型智能化绿色校园，获得了“湖南